In [0]:
import numpy as np
import pandas as pd
import matplotlib.image as img
from PIL import Image
from random import shuffle
import os, sys

def sigmoid(z):
    return 1/(1+np.exp(-z))

def reverse_sigmoid(z):
    return (sigmoid(z)*(1-sigmoid(z)))

def loss_entropy(x, y):
    return y*np.log(x+0.00000001)+(y-1)*np.log(1-x+0.00000001)

class Dense():
    def __init__(self, n_input, n_output, sigmoid=False, weights = None, prt=False):
        self.prt = prt
        self.n_input = n_input
        self.n_output = n_output
        self.sigmoid = sigmoid
        if weights == None:
            self.weights = np.random.randn(self.n_input, self.n_output)/100
        else:
            assert(weights.shape[0] == self.n_input and weights.shape[1] == self.n_output)
            self.weights = weights
        
    def forward(self, Z):
        assert(Z.shape[0])
        self.Z = Z
        if self.sigmoid:
            return sigmoid(np.dot(Z, self.weights))
        else: 
            return np.dot(Z, self.weights)
        
    def backward(self, loss, lr=0.01):
        assert(loss.shape[1] == self.n_output)
        if self.sigmoid:
            loss = reverse_sigmoid(np.dot(self.Z, self.weights))*loss
        if self.prt: print(self.weights, '/n')
        self.weights -= lr*np.dot(np.transpose(self.Z), loss)
        if self.prt:print(self.weights, '/n', lr*np.dot(np.transpose(self.Z), loss))
        return np.dot(loss, np.transpose(self.weights))
    
    def weights(self):
        return self.weights

class Filter():
    def __init__(self, dimension, matr_dim):
        self.filt = np.random.randn(dimension, dimension)
        self.dim = dimension
        self.matr_dim = matr_dim
        
    def forward(self, matrice):
        assert(matrice.shape[0] == self.matr_dim and matrice.shape[1] == self.matr_dim)
        self.matrice = matrice
        temp = np.zeros((self.matr_dim-self.dim+1, self.matr_dim-self.dim+1))
        for x in range(self.matr_dim-self.dim+1):
            for y in range(self.matr_dim-self.dim+1):
                for w in range(self.dim):
                    for h in range(self.dim):
                        temp[x][y] += matrice[x+w][y+h]*self.filt[w][h]
        return temp
    
    def backward(self, matr, lr):
        assert(matr.shape[0] == self.matrice.shape[0]-self.dim+1 and matr.shape[1] == matr.shape[0])
        diff = np.zeros((self.dim, self.dim))
        matr_diff = np.zeros((self.matr_dim, self.matr_dim))
        for i in range(matr.shape[0]):
            for j in range(matr.shape[1]):
                for x in range(self.dim):
                    for y in range(self.dim):
                        diff[x][y] += matr[i][j]*self.matrice[i+x][j+y]
                        matr_diff[i+x][j+y] = matr[i][j]*self.filt[x][y]
        self.filt -= lr*diff
        return matr_diff
                        

class Conv():
    def __init__(self, n_filter, n_matr, dim_filter, dim_matr):
        self.filters = []
        for i in range(n_matr):
            self.filters.append([Filter(dim_filter, dim_matr) for i in range(n_filter)])
        self.n_matr = n_matr
        self.n_filter = n_filter
        self.dim_matr = dim_matr
        self.dim_filter = dim_filter
        
    def forward(self, matrices):
        assert(len(matrices)==self.n_matr and len(matrices[0])==self.dim_matr and len(matrices[0][0])==self.dim_matr)
        new_shape = self.dim_matr-self.dim_filter+1
        result = np.array([np.zeros((new_shape, new_shape)) for i in range(self.n_filter)])
        for j, matr in enumerate(matrices):
            forw_matrs = np.array([filt.forward(matr) for filt in self.filters[j]])
            result += forw_matrs
        return result
    
    def backward(self, matrices, lr):
        shape = self.dim_matr-self.dim_filter+1
        assert(len(matrices) == self.n_filter and matrices[0].shape[0] == shape)
        result = np.array([np.zeros((self.dim_matr, self.dim_matr)) for i in range(self.n_matr)])
        for i, matr in enumerate(matrices):
            matr_diff = []
            for j in range(self.n_matr):
                matr_diff.append(self.filters[j][i].backward(matr, lr))
            matr_diff = np.array(matr_diff)
            result += matr_diff
        return result
    
class AvgPool():
    def __init__(self, dimension=2):
        self.dim = dimension
    
    def forward(self, matrices):
        assert(matrices[0].shape[0]%self.dim == 0)
        result = []
        self.new_shape = matrices[0].shape[0]//self.dim
        for matr in matrices:
            new_matr = np.zeros((self.new_shape, self.new_shape))
            for i in range(self.new_shape):
                for j in range(self.new_shape):
                    new_matr[i][j] = matr[i*self.dim:(i+1)*self.dim, j*self.dim:(j+1)*self.dim].mean()
            result.append(new_matr)
        return np.array(result)
        
    def backward(self, matrices):
        assert(matrices[0].shape[0] == self.new_shape)
        result = []
        for matr in matrices:
            big_matr = np.ones((self.new_shape*self.dim, self.new_shape*self.dim))*self.dim*self.dim
            for i in range(self.new_shape):
                for j in range(self.new_shape):
                    big_matr[i*self.dim:(i+1)*self.dim, j*self.dim:(j+1)*self.dim] *= matr[i][j]
            result.append(big_matr)
        return np.array(result)
      
class Model():
    def __init__(self):
        self.conv_1 = Conv(3, 3, 2, 35)
        self.conv_2 = Conv(3, 3, 2, 34)
        self.dense_1 = Dense(3267, 128, prt=False,)
        self.dense_2 = Dense(128, 10, sigmoid=True, prt=False)
        self.lrt = 0.0001
    
    def train(self, X, Y, lrt, epoch=1):
        assert(len(X)==len(Y))
        for _ in range(epoch):
            for i in range(len(X)):
                h = self.forward(X[i])
                loss = loss_entropy(h, Y[i])
                self.backward(loss)
    
    def forward(self, X):
        matrs = self.conv_1.forward(X)
        matrs_2 = self.conv_2.forward(matrs)
        matrs_3 = self.dense_1.forward(matrs_2.reshape(1, -1))
        matrs_4 = self.dense_2.forward(matrs_3)
        return matrs_4
  
    def backward(self, loss):
        loss_1 = self.dense_2.backward(loss, self.lrt)
        loss_2 = self.dense_1.backward(loss_1, self.lrt)
        loss_3 = loss_2.reshape((3, 33, 33))
        loss_4 = self.conv_2.backward(loss_3, self.lrt)
        loss_5 = self.conv_1.backward(loss_4, self.lrt)
    
    def predict(self, X):
        result = []
        for i in range(len(X)):
            res_vec = self.forward(X[i])
            maxim = 0
            res = 0
            for j in range(len(X[i])):
                if X[i][j] > maxim:
                    maxim = X[i][j]
                    res = j
            result.append(res)
        return result
    
asoebi = ['ვ', 'ჯ', 'შ', 'ნ', 'ს', 'ჟ', 'ე', 'ჩ', 'ო', 'ჭ']
appear = {'ვ':1467, 'ჯ':1070, 'შ':1331, 'ნ':1374, 'ს':1311, 'ჟ':1004, 'ე':1259, 'ჩ':1274, 'ო':1135, 'ჭ':1002}
path = './final_done/'
X = []
Y = []
X_te = []
Y_te = []
count = 0
for num, aso in enumerate(asoebi):
    currX = []
    currY = []
    for app in range(appear[aso]):
        im = Image.open(path+aso+'/'+aso+'_'+str(app)+'.jpg')
        x = np.array(im.resize((35, 35)).convert('RGB'))
        x = np.array([x[:,:,i] for i in range(3)])/255
        y = np.zeros(10)
        y[num] = 1
        y = y.reshape(1, -1)
        currX.append(x)
        currY.append(y)
        
    shuffle(currX)
    shuffle(currY)
    
    X_te.extend(currX[:200])
    Y_te.extend(currY[:200])
    X.extend(currX[200:])
    Y.extend(currY[200:])
      
X = np.array(X)
Y = np.array(Y)
X_te = np.array(X_te)
Y_te = np.array(Y_te)

In [0]:
model = Model()

In [0]:
for zdfz in range(5):    
    count = 0
    for tvla in range(4012,len(X)): 
        ra = np.random.randint(0, len(X))
        losa = model.forward(X[ra])
        loss = loss_entropy(losa, Y[ra])
        model.backward(loss)
        res = 0
        for i, re in enumerate(Y[ra][0]):
            if re == 1: res=i
        pas = 0
        shed = 0
        for i, re in enumerate(losa[0]):
            if re>shed:
                shed = re
                pas = i
        if pas == res:
            count += 1
        print(tvla, matrs_5, res, count, str(int(count/(tvla - 4011) * 10000) / 100) + "%" )

In [0]:
def get_prediction(modul, X_te):
    result = []
    for i in range(len(X_te)):
        res_vec = modul.forward(X_te[i])
        maxim = 0
        res = 0
        for j in range(len(res_vec[0])):
            if res_vec[0][j] > maxim:
                maxim = res_vec[0][j]
                res = j
        if maxim < 0.1:
            res = -1
        print(res_vec)
        result.append(res)
    return result

In [0]:
import pickle
# filehandler = open("savedModel2.pkl", 'wb') 
# pickle.dump(model, filehandler)

filehandler = open("savedModel1.pkl", 'rb') 
model1 = pickle.load(filehandler)

In [0]:
result = []
for i in range(200:220):
    res_vec = model1.forward(X_te[i])
    maxim = 0
    res = 0
    for j in range(len(res_vec)):
        if res_vec[0][j] > maxim:
            maxim = res_vec[0][j]
            res = j
    result.append(res)

KeyboardInterrupt: 

In [0]:
get_prediction(model1, [yl, yl])[0]

[[0.39482059 0.32726472 0.60174564 0.87836673 0.22272144 0.08822368
  0.52964715 0.79372871 0.57238789 0.18111308]]
[[0.39482059 0.32726472 0.60174564 0.87836673 0.22272144 0.08822368
  0.52964715 0.79372871 0.57238789 0.18111308]]


3

In [0]:
model1.forward(X_te[200])

array([[0.141386  , 0.72231507, 0.03119175, 0.00510686, 0.02955179,
        0.20344773, 0.04915413, 0.04484764, 0.04375665, 0.54786322]])

In [0]:
import csv
import os
from PIL import Image
import numpy as np

input_folder = './input/ე' #needs to be filled

def format_picture(image_path):
    image = Image.open(image_path)
    return image

def generate_ans(image):
    return 'a'

if __name__ == "__main__":
    image_files = os.listdir(input_folder)
    result = []
    for index in range(len(image_files)):
        img = format_picture('{}/{}'.format(input_folder, image_files[index]))
        ans = generate_ans(img)
        result += [[str(index), 'a']]

    filename = 'result.csv'
    with open(filename, 'w') as f:
        csv.writer(f).writerows(result)